In [ ]:
!pip list

In [ ]:
doc = {'text': 'Apple is looking at buying U.K. startup for $1 billion',
 'tokens': [{'i': 0,
   'text': 'Apple',
   'lemma_': 'Apple',
   'pos_': 'PROPN',
   'tag_': 'NNP',
   'dep_': 'nsubj',
   'shape_': 'Xxxxx',
   'is_alpha': True,
   'is_stop': False,
   'head': 2},
  {'i': 1,
   'text': 'is',
   'lemma_': 'be',
   'pos_': 'AUX',
   'tag_': 'VBZ',
   'dep_': 'aux',
   'shape_': 'xx',
   'is_alpha': True,
   'is_stop': True,
   'head': 2},
  {'i': 2,
   'text': 'looking',
   'lemma_': 'look',
   'pos_': 'VERB',
   'tag_': 'VBG',
   'dep_': 'ROOT',
   'shape_': 'xxxx',
   'is_alpha': True,
   'is_stop': False,
   'head': 2},
  {'i': 3,
   'text': 'at',
   'lemma_': 'at',
   'pos_': 'ADP',
   'tag_': 'IN',
   'dep_': 'prep',
   'shape_': 'xx',
   'is_alpha': True,
   'is_stop': True,
   'head': 2},
  {'i': 4,
   'text': 'buying',
   'lemma_': 'buy',
   'pos_': 'VERB',
   'tag_': 'VBG',
   'dep_': 'pcomp',
   'shape_': 'xxxx',
   'is_alpha': True,
   'is_stop': False,
   'head': 3},
  {'i': 5,
   'text': 'U.K.',
   'lemma_': 'U.K.',
   'pos_': 'PROPN',
   'tag_': 'NNP',
   'dep_': 'compound',
   'shape_': 'X.X.',
   'is_alpha': False,
   'is_stop': False,
   'head': 6},
  {'i': 6,
   'text': 'startup',
   'lemma_': 'startup',
   'pos_': 'NOUN',
   'tag_': 'NN',
   'dep_': 'dobj',
   'shape_': 'xxxx',
   'is_alpha': True,
   'is_stop': False,
   'head': 4},
  {'i': 7,
   'text': 'for',
   'lemma_': 'for',
   'pos_': 'ADP',
   'tag_': 'IN',
   'dep_': 'prep',
   'shape_': 'xxx',
   'is_alpha': True,
   'is_stop': True,
   'head': 4},
  {'i': 8,
   'text': '$',
   'lemma_': '$',
   'pos_': 'SYM',
   'tag_': '$',
   'dep_': 'quantmod',
   'shape_': '$',
   'is_alpha': False,
   'is_stop': False,
   'head': 10},
  {'i': 9,
   'text': '1',
   'lemma_': '1',
   'pos_': 'NUM',
   'tag_': 'CD',
   'dep_': 'compound',
   'shape_': 'd',
   'is_alpha': False,
   'is_stop': False,
   'head': 10},
  {'i': 10,
   'text': 'billion',
   'lemma_': 'billion',
   'pos_': 'NUM',
   'tag_': 'CD',
   'dep_': 'pobj',
   'shape_': 'xxxx',
   'is_alpha': True,
   'is_stop': False,
   'head': 7}]}

In [ ]:
from terminusdb_client import WOQLClient, WOQLQuery

db_id = "spacy_graph"
client = WOQLClient(server_url = "https://localhost:6363")
# client.connect(key="root", account="admin", user="admin")
# client.db(db_id)

In [ ]:
query = WOQLQuery().doctype("SpaCyToken")
.label("SpaCy Tokens")
.property("lemma", "string")
.property("pos", "string")
.property("tag", "string")
.property("dep", "string")
.property("shape", "string")
.property("is_alpha", "boolean")
.property("is_stop", "boolean")
.property("head", "SpaCyToken")
query.execute(client, "Create Schema")

In [ ]:
all_token = []
for token in doc:
    insert_token = (WOQLQuery().insert(f"v:token_id_self{token["i"]}", "scm:SpaCyToken", label=token.text)
                               .property("lemma", token["lemma_"])
                               .property("pos", token["pos_"])
                               .property("tag", token["tag_"])
                               .property("dep", token["dep_"])
                               .property("shape", token["shape_"])
                               .property("is_alpha", token["is_alpha"])
                               .property("is_stop", token["is_stop"])
                               .property("head", f"v:token_id_head{token['i']}")
                               .property("doc", f"v:token_doc_id{token['i']}")
                   )
    all_token.append(
                     WOQLQuery().idgen("doc:SpaCyToken",
                              [WOQLQuery().string(str(token["i"]))], f"v:token_id_self{token['i']}")
                     + WOQLQuery().idgen("doc:SpaCyToken",
                              [WOQLQuery().string(str(token["head"]["i"]))], f"v:token_id_head{token['i']}")
                     + insert_token
                     )
WOQLQuery().woql_and(*all_token).execute(client, "Add in all the tokens")